# Imports

In [1]:
import os
import torch
import random
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from utils.utils import load_parameters, stack_tensors
from datasets.datasets import load_data
# from inner_maximizers.inner_maximizers import inner_maximizer
from nets.ff_classifier import build_ff_classifier
# from blindspot_coverage.covering_number import CoveringNumber
import losswise
import time
import json
import numpy as np

# Loss Py < From https://github.com/xuanqing94/BayesianDefense >

In [2]:
import torch.nn.functional as F

def get_beta_backup(batch_idx, m, beta_type):
    if beta_type == "Blundell":
        beta = 2 ** (m - (batch_idx + 1)) / (2 ** m - 1)
    elif beta_type == "Soenderby":
        beta = min(epoch / (num_epochs // 4), 1)
    elif beta_type == "Standard":
        beta = 1 / m
    else:
        beta = 0
    return beta

AlphaPreDef = 1

def get_beta(epoch_idx, N):
    return AlphaPreDef / N

def elbo(out, y, kl_sum, beta):
    ce_loss = F.cross_entropy(out, y)
    return ce_loss + beta * kl_sum

# Inner Maximizers

In [3]:
# coding=utf-8
"""
Python module for implementing inner maximizers for robust adversarial training
(Table I in the paper)
"""
import torch
from torch.autograd import Variable
from utils.utils import or_float_tensors, xor_float_tensors, clip_tensor
import numpy as np


# helper function
def round_x(x, alpha=0.5):
    """
    rounds x by thresholding it according to alpha which can be a scalar or vector
    :param x:
    :param alpha: threshold parameter
    :return: a float tensor of 0s and 1s.
    """
    return (x > alpha).float()


def get_x0(x, is_sample=False):
    """
    Helper function to randomly initialize the the inner maximizer algos
    randomize such that the functionality is preserved.
    Functionality is preserved by maintaining the features present in x
    :param x: training sample
    :param is_sample: flag to sample randomly from feasible area or return just x
    :return: randomly sampled feasible version of x
    """
    if is_sample:
        rand_x = round_x(torch.rand(x.size()))
        if x.is_cuda:
            rand_x = rand_x.cuda()
        return or_float_tensors(x, rand_x)
    else:
        return x


def dfgsm_k(x,
            y,
            model,
            loss_fct,
            k=25,
            epsilon=0.02,
            alpha=0.5,
            is_report_loss_diff=True,
            use_sample=False):
    """
    FGSM^k with deterministic rounding
    :param y:
    :param x: (tensor) feature vector
    :param model: nn model
    :param loss_fct: loss function
    :param k: num of steps
    :param epsilon: update value in each direction
    :param alpha:
    :param is_report_loss_diff:
    :param use_sample:
    :return: the adversarial version of x according to dfgsm_k (tensor)
    """
    # some book-keeping
    if next(model.parameters()).is_cuda:
        x = x.cuda()
        y = y.cuda()
    y = Variable(y)

    # compute natural loss
    loss_natural = loss_fct(model(Variable(x)), y).data

    # initialize starting point
    x_next = get_x0(x, use_sample)

    # multi-step
    for t in range(k):
        # forward pass
        x_var = Variable(x_next, requires_grad=True)
        y_model = model(x_var)
        loss = loss_fct(y_model, y)

        # compute gradient
        grad_vars = torch.autograd.grad(loss.mean(), x_var)

        # find the next sample
        x_next = x_next + epsilon * torch.sign(grad_vars[0].data)

        # projection
        x_next = clip_tensor(x_next)

    # rounding step
    x_next = round_x(x_next, alpha=alpha)

    # feasible projection
    x_next = or_float_tensors(x_next, x)

    # compute adversarial loss
    loss_adv = loss_fct(model(Variable(x_next)), y).data

    if is_report_loss_diff:
        print("Natural loss (%.4f) vs Adversarial loss (%.4f), Difference: (%.4f)" %
              (loss_natural.mean(), loss_adv.mean(), loss_adv.mean() - loss_natural.mean()))

    replace_flag = (loss_adv < loss_natural).unsqueeze(1).expand_as(x_next)
    x_next[replace_flag] = x[replace_flag]

    if x_next.is_cuda:
        x_next = x_next.cpu()

    return x_next


def rfgsm_k(x, y, model, loss_fct, k=25, epsilon=0.02, is_report_loss_diff=True, use_sample=False):
    """
    FGSM^k with randomized rounding
    :param x: (tensor) feature vector
    :param y:
    :param model: nn model
    :param loss_fct: loss function
    :param k: num of steps
    :param epsilon: update value in each direction
    :param is_report_loss_diff:
    :param use_sample:
    :return: the adversarial version of x according to rfgsm_k (tensor)
    """
    # some book-keeping
    if next(model.parameters()).is_cuda:
        x = x.cuda()
        y = y.cuda()
    y = Variable(y)

#     print("\n\n======================\n===================")
#     print("RFGSM START")
#     print("=============")
#     print("x")
#     print(x)
#     print("=============")
#     print("y")
#     print(y)
#     print("=============")
#     print("model")
#     print(model)
#     print("=============")
#     print("loss_fct")
#     print(loss_fct)
#     print("=============")    
    # compute natural loss
    loss_natural = loss_fct(model(Variable(x)), y).data
#     print("loss_natural")
#     print(loss_natural)
#     print("=============")   
    # initialize starting point
    x_next = get_x0(x, use_sample)
#     print("x_next after init start pt")
#     print(x_next)
#     print("=============")  
    # multi-step with gradients
    for t in range(k):
        # forward pass
        x_var = Variable(x_next, requires_grad=True)
        y_model = model(x_var)
        loss = loss_fct(y_model, y)

        # compute gradient
        grad_vars = torch.autograd.grad(loss.mean(), x_var)

        # find the next sample
        x_next = x_next + epsilon * torch.sign(grad_vars[0].data)

        # projection
        x_next = clip_tensor(x_next)
#         print("x_next after multistep number : " + str(t))
#         print(x_next)
#         print("=============") 

    # rounding step
    alpha = torch.rand(x_next.size())
    if x_next.is_cuda:
        alpha = alpha.cuda()
    x_next = round_x(x_next, alpha=alpha)
#     print("x_next after rounding")
#     print(x_next)
#     print("=============")  

    # feasible projection
    x_next = or_float_tensors(x_next, x)
#     print("x_next after or float tensor")
#     print(x_next)
#     print("=============")  

    # compute adversarial loss
    loss_adv = loss_fct(model(Variable(x_next)), y).data
#     print("loss_adv = loss_fct(model(Variable(x_next)), y).data")
#     print(loss_adv)
#     print("=============")  

    if is_report_loss_diff:
        print("Natural loss (%.4f) vs Adversarial loss (%.4f), Difference: (%.4f)" %
              (loss_natural.mean(), loss_adv.mean(), loss_adv.mean() - loss_natural.mean()))

    replace_flag = (loss_adv < loss_natural).unsqueeze(1).expand_as(x_next)
    x_next[replace_flag] = x[replace_flag]
    print("replace_flag")
    print(replace_flag)
    print("=============")  
    print("x_next[replace_flag]")
    print(x_next[replace_flag])
    print("=============")  

    if x_next.is_cuda:
        x_next = x_next.cpu()

#     print("RFGSM END\n\n======================\n===================")
    return x_next


def bga_k(x, y, model, loss_fct, k=25, is_report_loss_diff=True, use_sample=False):
    """
    Multi-step bit gradient ascent
    :param x: (tensor) feature vector
    :param y:
    :param model: nn model
    :param loss_fct: loss function
    :param k: num of steps
    :param is_report_loss_diff:
    :param use_sample:
    :return: the adversarial version of x according to bga_k (tensor)
    """
    # some book-keeping
    sqrt_m = torch.from_numpy(np.sqrt([x.size()[1]])).float()

    if next(model.parameters()).is_cuda:
        x = x.cuda()
        y = y.cuda()
        sqrt_m = sqrt_m.cuda()

    y = Variable(y)

    # compute natural loss
    loss_natural = loss_fct(model(Variable(x)), y).data

    # keeping worst loss
    loss_worst = loss_natural.clone()
    x_worst = x.clone()

    # multi-step with gradients
    loss = None
    x_var = None
    x_next = None
    for t in range(k):
        if t == 0:
            # initialize starting point
            x_next = get_x0(x, use_sample)
        else:
            # compute gradient
            grad_vars = torch.autograd.grad(loss.mean(), x_var)
            grad_data = grad_vars[0].data

            # compute the updates
            x_update = (sqrt_m * (1. - 2. * x_next) * grad_data >= torch.norm(
                grad_data, 2, 1).unsqueeze(1).expand_as(x_next)).float()

            # find the next sample with projection to the feasible set
            x_next = xor_float_tensors(x_update, x_next)
            x_next = or_float_tensors(x_next, x)

        # forward pass
        x_var = Variable(x_next, requires_grad=True)
        y_model = model(x_var)
        loss = loss_fct(y_model, y)

        # update worst loss and adversarial samples
        replace_flag = (loss.data > loss_worst)
        loss_worst[replace_flag] = loss.data[replace_flag]
        x_worst[replace_flag.unsqueeze(1).expand_as(x_worst)] = x_next[replace_flag.unsqueeze(1)
                                                                       .expand_as(x_worst)]

    if is_report_loss_diff:
        print("Natural loss (%.4f) vs Adversarial loss (%.4f), Difference: (%.4f)" %
              (loss_natural.mean(), loss_worst.mean(), loss_worst.mean() - loss_natural.mean()))

    if x_worst.is_cuda:
        x_worst = x_worst.cpu()

    return x_worst


def bca_k(x, y, model, loss_fct, k=25, is_report_loss_diff=True, use_sample=False):
    """
    Multi-step bit coordinate ascent
    :param use_sample:
    :param is_report_loss_diff:
    :param y:
    :param x: (tensor) feature vector
    :param model: nn model
    :param loss_fct: loss function
    :param k: num of steps
    :return: the adversarial version of x according to bca_k (tensor)
    """
    if next(model.parameters()).is_cuda:
        x = x.cuda()
        y = y.cuda()

    y = Variable(y)

    # compute natural loss
    loss_natural = loss_fct(model(Variable(x)), y).data

    # keeping worst loss
    loss_worst = loss_natural.clone()
    x_worst = x.clone()

    # multi-step with gradients
    loss = None
    x_var = None
    x_next = None
    for t in range(k):
        if t == 0:
            # initialize starting point
            x_next = get_x0(x, use_sample)
        else:
            # compute gradient
            grad_vars = torch.autograd.grad(loss.mean(), x_var)
            grad_data = grad_vars[0].data

            # compute the updates (can be made more efficient than this)
            aug_grad = (1. - 2. * x_next) * grad_data
            val, _ = torch.topk(aug_grad, 1)
            x_update = (aug_grad >= val.expand_as(aug_grad)).float()

            # find the next sample with projection to the feasible set
            x_next = xor_float_tensors(x_update, x_next)
            x_next = or_float_tensors(x_next, x)

        # forward pass
        x_var = Variable(x_next, requires_grad=True)
        y_model = model(x_var)
        loss = loss_fct(y_model, y)

        # update worst loss and adversarial samples
        replace_flag = (loss.data > loss_worst)
        loss_worst[replace_flag] = loss.data[replace_flag]
        x_worst[replace_flag.unsqueeze(1).expand_as(x_worst)] = x_next[replace_flag.unsqueeze(1)
                                                                       .expand_as(x_worst)]

    if is_report_loss_diff:
        print("Natural loss (%.4f) vs Adversarial loss (%.4f), Difference: (%.4f)" %
              (loss_natural.mean(), loss_worst.mean(), loss_worst.mean() - loss_natural.mean()))

    if x_worst.is_cuda:
        x_worst = x_worst.cpu()

    return x_worst


def grosse_k(x, y, model, loss_fct, k=25, is_report_loss_diff=True, use_sample=False):
    """
    Multi-step bit coordinate ascent using gradient of output, advancing in direction of maximal change
    :param use_sample:
    :param is_report_loss_diff:
    :param loss_fct:
    :param y:
    :param x: (tensor) feature vector
    :param model: nn model
    :param k: num of steps
    :return adversarial version of x (tensor)
    """

    if next(model.parameters()).is_cuda:
        x = x.cuda()
        y = y.cuda()

    y = Variable(y)

    # compute natural loss
    loss_natural = loss_fct(model(Variable(x)), y).data

    # keeping worst loss
    loss_worst = loss_natural.clone()
    x_worst = x.clone()

    output = None
    x_var = None
    x_next = None
    for t in range(k):
        if t == 0:
            # initialize starting point
            x_next = get_x0(x, use_sample)
        else:
            grad_vars = torch.autograd.grad(output[:, 0].mean(), x_var)
            grad_data = grad_vars[0].data

            # Only consider gradients for points of 0 value
            aug_grad = (1. - x_next) * grad_data
            val, _ = torch.topk(aug_grad, 1)
            x_update = (aug_grad >= val.expand_as(aug_grad)).float()

            # find the next sample with projection to the feasible set
            x_next = xor_float_tensors(x_update, x_next)
            x_next = or_float_tensors(x_next, x)

        x_var = Variable(x_next, requires_grad=True)
        output = model(x_var)

        loss = loss_fct(output, y)

        # update worst loss and adversarial samples
        replace_flag = (loss.data > loss_worst)
        loss_worst[replace_flag] = loss.data[replace_flag]
        x_worst[replace_flag.unsqueeze(1).expand_as(x_worst)] = x_next[replace_flag.unsqueeze(1)
                                                                       .expand_as(x_worst)]

    if is_report_loss_diff:
        print("Natural loss (%.4f) vs Adversarial loss (%.4f), Difference: (%.4f)" %
              (loss_natural.mean(), loss_worst.mean(), loss_worst.mean() - loss_natural.mean()))

    if x_worst.is_cuda:
        x_worst = x_worst.cpu()

    return x_worst


def inner_maximizer(x, y, model, loss_fct, iterations=100, method='natural'):
    """
    A wrapper function for the above algorithim
    :param iterations:
    :param x:
    :param y:
    :param model:
    :param loss_fct:
    :param method: one of 'dfgsm_k', 'rfgsm_k', 'bga_k', 'bca_k', 'natural
    :return: adversarial examples
    """
    if method == 'dfgsm_k':
        return dfgsm_k(x, y, model, loss_fct, k=iterations)
    elif method == 'rfgsm_k':
        return rfgsm_k(x, y, model, loss_fct, k=iterations)
    elif method == 'bga_k':
        return bga_k(x, y, model, loss_fct, k=iterations)
    elif method == 'bca_k':
        return bca_k(x, y, model, loss_fct, k=iterations)
    elif method == 'grosse':
        return grosse_k(x, y, model, loss_fct, k=iterations)
    elif method == 'natural':
        return x
    else:
        raise Exception('No such inner maximizer algorithm')


# Parameter Loads

In [4]:
parameters = load_parameters("parameters.ini")
is_cuda = eval(parameters["general"]["is_cuda"])
if is_cuda:
    os.environ["CUDA_VISIBLE_DEVICES"] = parameters["general"]["gpu_device"]

assertion_message = "Set this flag off to train models."
assert eval(parameters['dataset']['generate_feature_vector_files']) is False, assertion_message

log_interval = int(parameters["general"]["log_interval"])
num_epochs = int(parameters["hyperparam"]["ff_num_epochs"])
is_losswise = eval(parameters["general"]["is_losswise"])
is_synthetic_dataset = eval(parameters["general"]["is_synthetic_dataset"])

training_method = parameters["general"]["training_method"]
evasion_method = parameters["general"]["evasion_method"]
experiment_suffix = parameters["general"]["experiment_suffix"]
experiment_name = "[training:%s|evasion:%s]_%s" % (training_method, evasion_method,
                                                   experiment_suffix)

print("Training Method:%s, Evasion Method:%s" % (training_method, evasion_method))

seed_val = int(parameters["general"]["seed"])

random.seed(seed_val)
torch.manual_seed(seed_val)
np.random.seed(seed_val)


evasion_iterations = int(parameters['hyperparam']['evasion_iterations'])

save_every_epoch = eval(parameters['general']['save_every_epoch'])

train_model_from_scratch = eval(parameters['general']['train_model_from_scratch'])
load_model_weights = eval(parameters['general']['load_model_weights'])
model_weights_path = parameters['general']['model_weights_path']

Training Method:rfgsm_k, Evasion Method:rfgsm_k


# Data Load

In [5]:
train_dataloader_dict, valid_dataloader_dict, test_dataloader_dict, num_features = load_data(
    parameters)

Starting data loading
Creating import to index mapping
-- accessing file: ./helper_files/N5000/pmal/
-- accessing file: ./helper_files/N5000/pben/
Malicious files: 500
Benign files: 500
Total number of files: 1000
0 Time: 0.0  seconds
100 Time: 0.0904228687286377  seconds


Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Deskto

Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Deskto

Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Deskto

Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Deskto

...

900 Time: 0.863142728805542  seconds


Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Deskto

Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Deskto

-- accessing file: ./helper_files/None.txt
-- accessing file: ./helper_files/N5000/pmal/
-- accessing file: ./helper_files/N5000/pben/
Malware Files: 100
Benign Files: 100
Preparing training datasets
Preparing validation datasets
Preparing testing datasets
FAL
./helper_files/N5000/pmal/Pickle_malware118.pt
FV
tensor([0., 0., 0.,  ..., 0., 0., 0.])
FAL
./helper_files/N5000/pben/Pickle_benign105.pt
FV
tensor([0., 0., 0.,  ..., 0., 0., 0.])
FAL
./helper_files/N5000/pmal/Pickle_malware170.pt
FV
tensor([0., 0., 0.,  ..., 0., 0., 0.])
FAL
./helper_files/N5000/pben/Pickle_benign153.pt
FV
tensor([0., 0., 0.,  ..., 0., 0., 0.])
FAL
./helper_files/N5000/pmal/Pickle_malware170.pt
FV
tensor([0., 0., 0.,  ..., 0., 0., 0.])
FAL
./helper_files/N5000/pben/Pickle_benign105.pt
FV
tensor([0., 0., 0.,  ..., 0., 0., 0.])
FAL
./helper_files/N5000/pmal/Pickle_malware118.pt
FV
tensor([0., 0., 0.,  ..., 0., 0., 0.])
FAL
./helper_files/N5000/pben/Pickle_benign153.pt
FV
tensor([0., 0., 0.,  ..., 0., 0., 0.])
FAL

Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Deskto

# Coverage

In [6]:
num_samples = len(train_dataloader_dict["malicious"].dataset)

In [7]:
# import pybloomfilter
import numpy
import random

class Coverage(object):

    def __init__(self, num_samples, error_rate=0.000001):
        self._num_samples = num_samples
        self._expected_num_points = num_samples
        self._denominator = self._expected_num_points
        self._numerator = 0.0

#         self._bf = pybloomfilter.BloomFilter(self._expected_num_points, error_rate)
        self._bf = []

        print("\n\n======================\n===================")
        print("INIT START")
        print("=============")
        print("_num_samples")
        print(self._num_samples)
        print("=============")
        print("_expected_num_points")
        print(self._expected_num_points)
        print("=============")
        print("INIT END\n\n======================\n===================")

    def update(self, point):

#         print("\n\n======================\n===================")
#         print("UPDATE NUMERATOR START")
#         print("=============")
#         print("point")
#         print(point)
#         print("=============")
        pt_np = point.numpy().tolist()
#         is_not_in = not self._bf.add(hash(str(pt_np)))
#         self._numerator += int(is_not_in)
        if str(pt_np) not in self._bf:
            self._numerator += 1
            self._bf.append(str(pt_np))
            print("NEW POINT")
        print("=============")
#         print("is_not_in")
#         print(is_not_in)
#         print("=============")
#         print("self._numerator")
#         print(self._numerator)
#         print("=============")
#         print("UPDATE NUMERATOR END\n\n======================\n===================")
        return(self._numerator/(self._num_samples))
        
    def Covering_value(self):
        """
        :return: the ratio of the visited samples to the maximum expected ones
        """
        print("\n\n======================\n===================")
        print("RATIO START")
        print("=============")
        print("self._actual_num_points")
        print(self._numerator)
        print("=============")
        print("self._expected_num_points")
        print(self._denominator)
        print("=============")
        print("self._actual_num_points * 1. / self._expected_num_points")
        print(self._numerator * 1. / self._denominator)
        print("=============")
        print("RATIO END\n\n======================\n===================")
        return self._numerator * 1. / self._denominator

In [8]:
bscn = Coverage(num_samples)
gscn = Coverage(num_samples)



INIT START
_num_samples
60
_expected_num_points
60
INIT END



INIT START
_num_samples
60
_expected_num_points
60
INIT END



# Model Build , Loss and Optim

In [9]:
model = build_ff_classifier(
        input_size=num_features,
        hidden_1_size=int(parameters["hyperparam"]["ff_h1"]),
        hidden_2_size=int(parameters["hyperparam"]["ff_h2"]),
        hidden_3_size=int(parameters["hyperparam"]["ff_h3"]))
model

Sequential(
  (0): Linear(in_features=2948, out_features=300, bias=True)
  (1): ReLU()
  (2): Linear(in_features=300, out_features=300, bias=True)
  (3): ReLU()
  (4): Linear(in_features=300, out_features=300, bias=True)
  (5): ReLU()
  (6): Linear(in_features=300, out_features=2, bias=True)
  (7): LogSoftmax(dim=1)
)

In [10]:
loss_fct = nn.NLLLoss(reduce=False)
optimizer = optim.Adam(model.parameters(), lr=float(parameters["hyperparam"]["ff_learning_rate"]))
optimizer

C:\ProgramData\Anaconda3\envs\nn_mal_2\lib\site-packages\torch\nn\_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

# Train Test Validate

In [11]:
# def train_backup(epoch):
#     model.train()
#     total_correct = 0.
#     total_loss = 0.
#     total = 0.

#     current_time = time.time()

#     if is_synthetic_dataset:
#         # since generation of synthetic data set is random, we'd like them to be the same over epochs
#         torch.manual_seed(seed_val)
#         random.seed(seed_val)
#     print("dataloaderr",train_dataloader_dict["benign"].dataset, train_dataloader_dict["malicious"].dataset)
#     for batch_idx, ((bon_x, bon_y), (mal_x, mal_y)) in enumerate(
#             zip(train_dataloader_dict["benign"], train_dataloader_dict["malicious"])):
#         # Check for adversarial learning
# #         print("This is bacth_idx",batch_idx)
# #         print(bon_x)
# #         print(bon_y)
# #         print(mal_x)
# #         print(mal_y)
#         mal_x1 = inner_maximizer(
#             mal_x, mal_y, model, loss_fct, iterations=evasion_iterations, method=training_method)
# #         print("THis is mal_X",mal_x1)
#         # stack input
#         if is_cuda:
#             x = Variable(stack_tensors(bon_x, mal_x, mal_x1).cuda())
#             y = Variable(stack_tensors(bon_y, mal_y, mal_y).cuda())
#         else:
#             x = Variable(stack_tensors(bon_x, mal_x, mal_x1))
#             y = Variable(stack_tensors(bon_y, mal_y, mal_y))
# #         print("STACKED")
# #         print(x)
# #         print(y)
# #         print("This is Train Forward Pass",x, y, model )
        
#         # forward pass
# #         print("FORWARD X")
# #         print(x)
#         y_model = model(x)

#         # backward pass
#         optimizer.zero_grad()
#         loss = loss_fct(y_model, y).mean()
#         loss.backward()
#         optimizer.step()

#         # predict pass
#         _, predicted = torch.topk(y_model, k=1)
#         correct = predicted.data.eq(y.data.view_as(predicted.data)).cpu().sum()

#         # metrics
# #         print("LOSS LINE 148")
# #         print(len(y))
# #         print(y)
# #         print(total_loss)
# #         print(loss.data)
# #         print(loss.data.item())
#         # total_loss += loss.data[0] * len(y)
#         total_loss += loss.data.item() * len(y)

#         total_correct += correct
#         total += len(y)

# #         bscn.update_numerator_batch(batch_idx, mal_x)
#         print("COVERING UPDATES",mal_x.size(0))
#         for i in range(mal_x.size(0)):
#             print("UPDATING NORMAL MALS")
#             a = bscn.update(mal_x[i])
#             print("UPDATINGS ADVS")
#             b = bscn.update(mal_x1[i])
#             print("UPDATING GOOD SPOTS")
#             c = gscn.update(bon_x[i])
#             print(a,b,c)
        

#         if 15%5 == 0:

#             print("Time Taken:", time.time() - current_time)
#             current_time = time.time()

#             print(
#                 "Train Epoch ({}) | Batch ({}) | [{}/{} ({:.0f}%)]\tBatch Loss: {:.6f}\tBatch Accuracy: {:.1f}%\t BSCN: {:.12f}".
#                 # format(epoch, batch_idx, batch_idx * len(x),
#                 #        len(train_dataloader_dict["malicious"].dataset) +
#                 #        len(train_dataloader_dict["benign"].dataset),
#                 #        100. * batch_idx / len(train_dataloader_dict["benign"]), loss.data[0],
#                 #        100. * correct / len(y), bscn.ratio()))
#                 format(epoch, batch_idx, batch_idx * len(x),
#                        len(train_dataloader_dict["malicious"].dataset) +
#                        len(train_dataloader_dict["benign"].dataset),
#                        100. * batch_idx / len(train_dataloader_dict["benign"]), loss.data.item(),
#                        100. * correct / len(y), bscn.Covering_value()))


In [12]:
def train(epoch):
    model.train()
    total_correct = 0.
    total_loss = 0.
    total = 0.

    current_time = time.time()

    if is_synthetic_dataset:
        # since generation of synthetic data set is random, we'd like them to be the same over epochs
        torch.manual_seed(seed_val)
        random.seed(seed_val)
    print("dataloaderr",train_dataloader_dict["benign"].dataset, train_dataloader_dict["malicious"].dataset)
    for batch_idx, ((bon_x, bon_y), (mal_x, mal_y)) in enumerate(
            zip(train_dataloader_dict["benign"], train_dataloader_dict["malicious"])):
        # Check for adversarial learning
#         print("This is bacth_idx",batch_idx)
#         print(bon_x)
#         print(bon_y)
#         print(mal_x)
#         print(mal_y)
        mal_x1 = inner_maximizer(
            mal_x, mal_y, model, loss_fct, iterations=evasion_iterations, method=training_method)
#         print("THis is mal_X",mal_x1)
        # stack input
        if is_cuda:
            x = Variable(stack_tensors(bon_x, mal_x, mal_x1).cuda())
            y = Variable(stack_tensors(bon_y, mal_y, mal_y).cuda())
        else:
            x = Variable(stack_tensors(bon_x, mal_x, mal_x1))
            y = Variable(stack_tensors(bon_y, mal_y, mal_y))
#         print("STACKED")
#         print(x)
#         print(y)
#         print("This is Train Forward Pass",x, y, model )
        
        # forward pass
#         print("FORWARD X")
#         print(x)
#         y_model = model(x)

#         # backward pass
#         optimizer.zero_grad()
#         loss = loss_fct(y_model, y).mean()
#         loss.backward()
#         optimizer.step()

        # Training by Bayes by Backprop
        optimizer.zero_grad()
        y_model = model(x)
        kl = loss_fct(y_model, y).mean()
        loss = elbo(y_model, y, kl, 
                    get_beta(epoch,(train_dataloader_dict["benign"].dataset.__len__() 
                                    + 2*train_dataloader_dict["malicious"].dataset.__len__()) ))
        loss.backward()
        optimizer.step()

        # predict pass
        _, predicted = torch.topk(y_model, k=1)
        correct = predicted.data.eq(y.data.view_as(predicted.data)).cpu().sum()

        # metrics
#         print("LOSS LINE 148")
#         print(len(y))
#         print(y)
#         print(total_loss)
#         print(loss.data)
#         print(loss.data.item())
        # total_loss += loss.data[0] * len(y)
        total_loss += loss.data.item() * len(y)

        total_correct += correct
        total += len(y)

#         bscn.update_numerator_batch(batch_idx, mal_x)
        print("COVERING UPDATES",mal_x.size(0))
        for i in range(mal_x.size(0)):
            print("UPDATING NORMAL MALS")
            a = bscn.update(mal_x[i])
            print("UPDATINGS ADVS")
            b = bscn.update(mal_x1[i])
            print("UPDATING GOOD SPOTS")
            c = gscn.update(bon_x[i])
            print(a,b,c)
        

        if 15%5 == 0:

            print("Time Taken:", time.time() - current_time)
            current_time = time.time()

            print(
                "Train Epoch ({}) | Batch ({}) | [{}/{} ({:.0f}%)]\tBatch Loss: {:.6f}\tBatch Accuracy: {:.1f}%\t BSCN: {:.12f}".
                # format(epoch, batch_idx, batch_idx * len(x),
                #        len(train_dataloader_dict["malicious"].dataset) +
                #        len(train_dataloader_dict["benign"].dataset),
                #        100. * batch_idx / len(train_dataloader_dict["benign"]), loss.data[0],
                #        100. * correct / len(y), bscn.ratio()))
                format(epoch, batch_idx, batch_idx * len(x),
                       len(train_dataloader_dict["malicious"].dataset) +
                       len(train_dataloader_dict["benign"].dataset),
                       100. * batch_idx / len(train_dataloader_dict["benign"]), loss.data.item(),
                       100. * correct / len(y), bscn.Covering_value()))


In [13]:
def check_one_category(category="benign", is_validate=False, is_evade=False,
                       evade_method='dfgsm_k'):
    """
    test the model in terms of loss and accuracy on category, this function also allows to perform perturbation
    with respect to loss to evade
    :param category: benign or malicious dataset
    :param is_validate: validation or testing dataset
    :param is_evade: to perform evasion or not
    :param evade_method: evasion method (we can use on of the inner maximier methods), it is only relevant if is_evade
      is True
    :return:
    """
    model.eval()
    total_loss = 0
    total_correct = 0
    total = 0
    evasion_mode = ""

    if is_synthetic_dataset:
        # since generation of synthetic data set is random, we'd like them to be the same over epochs
        torch.manual_seed(seed_val)
        random.seed(seed_val)

    if is_validate:
        dataloader = valid_dataloader_dict[category]
    else:
        dataloader = test_dataloader_dict[category]

    for batch_idx, (x, y) in enumerate(dataloader):
        #
        if is_evade:
            x = inner_maximizer(
                x, y, model, loss_fct, iterations=evasion_iterations, method=evade_method)
            evasion_mode = "(evasion using %s)" % evade_method
        # stack input
        if is_cuda:
            x = Variable(x.cuda())
            y = Variable(y.cuda())
        else:
            x = Variable(x)
            y = Variable(y)
#         print("Check one CategoryForward Pass", x, y, model)
        # forward pass
        y_model = model(x)

        # loss pass
        loss = loss_fct(y_model, y).mean()

        # predict pass
        _, predicted = torch.topk(y_model, k=1)
        correct = predicted.data.eq(y.data.view_as(predicted.data)).cpu().sum()

        # metrics
        # total_loss += loss.data[0] * len(y)
        total_loss += loss.data.item() * len(y)
        total_correct += correct
        total += len(y)

    print("{} set for {} {}: Average Loss: {:.4f}, Accuracy: {:.2f}%".format(
        "Valid" if is_validate else "Test", category, evasion_mode, total_loss / total,
        total_correct * 100. / total))

    return total_loss, total_correct, total

In [14]:
def test(epoch, is_validate=False):
    """
    Function to be used for both testing and validation
    :param epoch: current epoch
    :param is_validate: is the testing done on the validation dataset
    :return: average total loss, dictionary of the metrics for both bon and mal samples
    """
    # test for accuracy and loss
    bon_total_loss, bon_total_correct, bon_total = check_one_category(
        category="benign", is_evade=False, is_validate=is_validate)
    mal_total_loss, mal_total_correct, mal_total = check_one_category(
        category="malicious", is_evade=False, is_validate=is_validate)

    # test for evasion on malicious sample
    evade_mal_total_loss, evade_mal_total_correct, evade_mal_total = check_one_category(
        category="malicious", is_evade=True, evade_method=evasion_method, is_validate=is_validate)

    total_loss = bon_total_loss + mal_total_loss
    total_correct = bon_total_correct + mal_total_correct
    total = bon_total + mal_total

    dataset_type = "valid" if is_validate else "test"

    print("{} set overall: Average Loss: {:.4f}, Accuracy: {:.2f}%".format(
        dataset_type, total_loss / total, total_correct * 100. / total))

    if is_losswise:
        graph_accuracy.append(
            epoch, {
                "%s_accuracy_%s" % (dataset_type, experiment_name): 100. * total_correct / total
            })
        graph_loss.append(epoch, {
            "%s_loss_%s" % (dataset_type, experiment_name): total_loss / total
        })
        graph_evasion.append(
            epoch, {
                "%s_evasion_%s" % (dataset_type, experiment_name):
                100 * (evade_mal_total - evade_mal_total_correct) / evade_mal_total
            })

    metrics = {
        "bscn_ratio": bscn.Covering_value(),
        "mal": {
            "total_loss": mal_total_loss,
            "total_correct": mal_total_correct,
            "total": mal_total,
            "evasion": {
                "total_loss": evade_mal_total_loss,
                "total_correct": evade_mal_total_correct,
                "total": evade_mal_total
            }
        },
        "bon": {
            "total_loss": bon_total_loss,
            "total_correct": bon_total_correct,
            "total_evade": None,
            "total": bon_total
        }
    }
    print(metrics)

    return (bon_total_loss + max(mal_total_loss, evade_mal_total_loss)) / total, metrics


In [15]:
if __name__ == "__main__":

    if not os.path.exists("result_files"):
        os.mkdir("result_files")

    _metrics = None
    session = None
    if train_model_from_scratch:
        best_valid_loss = float("inf")
        for _epoch in range(10):
            # train
            train(_epoch)

            # validate
            valid_loss, _ = test(_epoch, is_validate=True)

            # keep the best parameters w.r.t validation and check the test set
            if best_valid_loss > valid_loss:
                best_valid_loss = valid_loss
                _, _metrics = test(_epoch, is_validate=False)

                bscn_to_save = bscn.Covering_value()
                # with open(os.path.join("result_files", "%s_bscn_test.txt" % experiment_name), "w") as f:
                #     f.write(str(bscn_to_save))
                print("BSCN SAVING")
                print(bscn_to_save)

                # torch.save(model, os.path.join("helper_files", "%s-model.pt" % experiment_name))
            elif _epoch % log_interval == 0:
                test(_epoch, is_validate=False)

    else:
        _, _metrics = test(0)

    # with open(os.path.join("result_files", experiment_name + ".json"), "w") as result_file:
    #     json.dump(_metrics, result_file)
    print("METRICS TO DUMP")
    print(_metrics)


dataloaderr <datasets.datasets.PortableExecutableDataset object at 0x000001CF0B40A630> <datasets.datasets.PortableExecutableDataset object at 0x000001CF084FE080>
Natural loss (0.6895) vs Adversarial loss (0.6999), Difference: (0.0104)
replace_flag
tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])
x_next[replace_flag]
tensor([])
COVERING UPDATES 8
UPDATING NORMAL MALS
NEW POINT
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
NEW POINT
0.016666666666666666 0.03333333333333333 0.016666666666666666
UPDATING NORMAL MALS
NEW POINT
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
NEW POINT
0.05 0.06666666666666667 0.03333333333333333
UPDATING NORMAL MALS
NEW POINT
UPDATINGS ADVS
NEW POINT
UPDA

NEW POINT
UPDATING GOOD SPOTS
NEW POINT
1.15 1.1666666666666667 0.6166666666666667
UPDATING NORMAL MALS
NEW POINT
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
NEW POINT
1.1833333333333333 1.2 0.6333333333333333
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
NEW POINT
1.2 1.2166666666666666 0.65
UPDATING NORMAL MALS
NEW POINT
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
NEW POINT
1.2333333333333334 1.25 0.6666666666666666
Time Taken: 0.07679557800292969


RATIO START
self._actual_num_points
75.0
self._expected_num_points
60
self._actual_num_points * 1. / self._expected_num_points
1.25
RATIO END

Train Epoch (0) | Batch (4) | [96/120 (50%)]	Batch Loss: 0.641378	Batch Accuracy: 66.7%	 BSCN: 1.250000000000
Natural loss (0.5078) vs Adversarial loss (0.5293), Difference: (0.0215)
replace_flag
tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        .

Valid set for malicious (evasion using rfgsm_k): Average Loss: 0.3106, Accuracy: 100.00%
valid set overall: Average Loss: 0.6795, Accuracy: 50.00%


RATIO START
self._actual_num_points
112.0
self._expected_num_points
60
self._actual_num_points * 1. / self._expected_num_points
1.8666666666666667
RATIO END

{'bscn_ratio': 1.8666666666666667, 'mal': {'total_loss': 5.524043917655945, 'total_correct': tensor(20), 'total': 20, 'evasion': {'total_loss': 6.2115278244018555, 'total_correct': tensor(20), 'total': 20}}, 'bon': {'total_loss': 21.656912326812744, 'total_correct': tensor(0), 'total_evade': None, 'total': 20}}
Test set for benign : Average Loss: 1.0314, Accuracy: 0.00%
Test set for malicious : Average Loss: 0.2925, Accuracy: 100.00%
Natural loss (0.2794) vs Adversarial loss (0.3144), Difference: (0.0350)
replace_flag
tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, Fal

UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
2.2666666666666666 2.283333333333333 0.9833333333333333
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
2.283333333333333 2.3 0.9833333333333333
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
2.3 2.316666666666667 0.9833333333333333
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
2.316666666666667 2.3333333333333335 0.9833333333333333
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
2.3333333333333335 2.35 0.9833333333333333
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
2.35 2.3666666666666667 0.9833333333333333
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
2.3666666666666667 2.3833333333333333 0.9833333333333333
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
2.3833333333333333 2.4 0.9833333333333333
Time Taken: 0.10272645950317383


RATIO START
self._actual_num_points
144.0
self._expected_num_points
60
self._act

Valid set for benign : Average Loss: 0.6131, Accuracy: 70.00%
Valid set for malicious : Average Loss: 0.2125, Accuracy: 90.00%
Natural loss (0.0630) vs Adversarial loss (0.1510), Difference: (0.0880)
replace_flag
tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])
x_next[replace_flag]
tensor([])
Natural loss (0.4149) vs Adversarial loss (0.5032), Difference: (0.0883)
replace_flag
tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
    

...

COVERING UPDATES 8
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
8.266666666666667 8.283333333333333 0.9833333333333333
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
8.283333333333333 8.3 0.9833333333333333
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
8.3 8.316666666666666 0.9833333333333333
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
8.316666666666666 8.333333333333334 0.9833333333333333
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
8.333333333333334 8.35 0.9833333333333333
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
8.35 8.366666666666667 0.9833333333333333
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
8.366666666666667 8.383333333333333 0.9833333333333333
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
8.383333333333333 8.4 0.9833333333333333
Time Taken: 0.0718085765838623


RATIO START
self._actual_num_points
504.0
self._

Valid set for benign : Average Loss: 0.7761, Accuracy: 65.00%
Valid set for malicious : Average Loss: 0.0681, Accuracy: 95.00%
Natural loss (0.0118) vs Adversarial loss (0.2271), Difference: (0.2152)
replace_flag
tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])
x_next[replace_flag]
tensor([])
Natural loss (0.1582) vs Adversarial loss (0.6864), Difference: (0.5282)
replace_flag
tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
    

COVERING UPDATES 8
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
9.4 9.416666666666666 0.9833333333333333
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
9.416666666666666 9.433333333333334 0.9833333333333333
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
9.433333333333334 9.45 0.9833333333333333
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
9.45 9.466666666666667 0.9833333333333333
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
9.466666666666667 9.483333333333333 0.9833333333333333
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
9.483333333333333 9.5 0.9833333333333333
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
9.5 9.516666666666667 0.9833333333333333
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
9.516666666666667 9.533333333333333 0.9833333333333333
Time Taken: 0.07878851890563965


RATIO START
self._actual_num_points
572.0
self.

Valid set for malicious (evasion using rfgsm_k): Average Loss: 0.6676, Accuracy: 85.00%
valid set overall: Average Loss: 0.3582, Accuracy: 85.00%


RATIO START
self._actual_num_points
592.0
self._expected_num_points
60
self._actual_num_points * 1. / self._expected_num_points
9.866666666666667
RATIO END

{'bscn_ratio': 9.866666666666667, 'mal': {'total_loss': 4.998270924203098, 'total_correct': tensor(18), 'total': 20, 'evasion': {'total_loss': 13.351345654577017, 'total_correct': tensor(17), 'total': 20}}, 'bon': {'total_loss': 9.330719888210297, 'total_correct': tensor(16), 'total_evade': None, 'total': 20}}
dataloaderr <datasets.datasets.PortableExecutableDataset object at 0x000001CF0B40A630> <datasets.datasets.PortableExecutableDataset object at 0x000001CF084FE080>
Natural loss (0.0026) vs Adversarial loss (0.0177), Difference: (0.0150)
replace_flag
tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, Fal

COVERING UPDATES 8
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
10.666666666666666 10.683333333333334 0.9833333333333333
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
10.683333333333334 10.7 0.9833333333333333
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
10.7 10.716666666666667 0.9833333333333333
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
10.716666666666667 10.733333333333333 0.9833333333333333
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
10.733333333333333 10.75 0.9833333333333333
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
10.75 10.766666666666667 0.9833333333333333
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
10.766666666666667 10.783333333333333 0.9833333333333333
UPDATING NORMAL MALS
UPDATINGS ADVS
NEW POINT
UPDATING GOOD SPOTS
10.783333333333333 10.8 0.9833333333333333
Time Taken: 0.05884265899658203


RATIO START
self._actual_num_po